In [68]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

In [69]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

train.shape, test.shape

((256351, 68), (90067, 67))

In [70]:
drop_columns2 = []
for col in train.columns:
  print(train[col].value_counts())
  if len(train[col].value_counts()) == 1:
    drop_columns2.append(col)

시술 시기 코드
TRDQAZ    38969
TRCMWS    38090
TRYBLT    36713
TRVNRY    36173
TRJXFG    36031
TRZKPL    35544
TRXQMD    34831
Name: count, dtype: int64
시술 당시 나이
만18-34세    102476
만35-37세     57780
만38-39세     39247
만40-42세     37348
만43-44세     12253
만45-50세      6918
알 수 없음        329
Name: count, dtype: int64
임신 시도 또는 마지막 임신 경과 연수
7.0     1214
8.0     1176
9.0     1023
6.0      991
10.0     947
11.0     701
5.0      692
12.0     510
13.0     481
14.0     331
4.0      305
15.0     254
16.0     202
17.0     140
18.0     106
3.0       89
19.0      86
20.0      83
2.0       21
0.0       12
1.0        6
Name: count, dtype: int64
시술 유형
IVF    250060
DI       6291
Name: count, dtype: int64
특정 시술 유형
ICSI                                   122368
IVF                                     91755
Unknown                                 26939
IUI                                      6100
ICSI:ICSI                                2314
ICSI / BLASTOCYST                        1609
IVF / BLASTOCYST          

In [71]:
for col in test.columns:
  print(test[col].value_counts())

시술 시기 코드
TRDQAZ    13610
TRCMWS    13392
TRYBLT    12929
TRZKPL    12793
TRJXFG    12764
TRVNRY    12560
TRXQMD    12019
Name: count, dtype: int64
시술 당시 나이
만18-34세    36178
만35-37세    20211
만38-39세    13611
만40-42세    13266
만43-44세     4272
만45-50세     2413
알 수 없음       116
Name: count, dtype: int64
임신 시도 또는 마지막 임신 경과 연수
7.0     405
8.0     403
6.0     352
9.0     320
10.0    311
5.0     286
11.0    253
12.0    205
13.0    168
14.0    127
4.0     106
15.0     92
16.0     64
17.0     64
18.0     44
20.0     33
19.0     29
3.0      23
2.0       5
0.0       4
1.0       3
Name: count, dtype: int64
시술 유형
IVF    87891
DI      2176
Name: count, dtype: int64
특정 시술 유형
ICSI                                  43237
IVF                                   32350
Unknown                                9216
IUI                                    2112
ICSI:ICSI                               797
ICSI / BLASTOCYST                       584
IVF:IVF                                 406
IVF / BLASTOCYST        

In [72]:
print(drop_columns2)
train.drop(columns=drop_columns2, inplace=True)
test.drop(columns=drop_columns2, inplace=True)

train.shape, test.shape

['착상 전 유전 검사 사용 여부', '불임 원인 - 여성 요인', 'PGD 시술 여부', 'PGS 시술 여부', '난자 채취 경과일']


((256351, 63), (90067, 62))

In [73]:
cat_to_int_dict = {'6회 이상' : 6, '5회' : 5, '4회' : 4, '3회' : 3, '2회' : 2, '1회' : 1, '0회' : 0}

cat_to_int_columns = ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수','DI 시술 횟수',
                      '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수','DI 출산 횟수']
for col in cat_to_int_columns:
  train[col] = train[col].map(cat_to_int_dict)
  test[col] = test[col].map(cat_to_int_dict)

In [74]:
train.loc[train['불임 원인 - 정자 농도'] == 1]['임신 성공 여부'].value_counts()

임신 성공 여부
0    207
1     69
Name: count, dtype: int64

In [75]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# categorical_columns = ["시술 시기 코드","시술 당시 나이", "시술 유형", "특정 시술 유형","배란 자극 여부",
                      #  "단일 배아 이식 여부","착상 전 유전 진단 사용 여부",
                      #  "남성 주 불임 원인","남성 부 불임 원인","여성 주 불임 원인","여성 부 불임 원인",
                      #  "부부 주 불임 원인","부부 부 불임 원인","불명확 불임 원인","불임 원인 - 난관 질환",
                      #  "불임 원인 - 남성 요인","불임 원인 - 배란 장애","불임 원인 - 자궁경부 문제","불임 원인 - 자궁내막증",
                      #  "불임 원인 - 정자 농도","불임 원인 - 정자 면역학적 요인","불임 원인 - 정자 운동성","불임 원인 - 정자 형태",
                      #  "배아 생성 주요 이유","총 시술 횟수","클리닉 내 총 시술 횟수","IVF 시술 횟수","DI 시술 횟수",
                      #  "총 임신 횟수","IVF 임신 횟수","DI 임신 횟수","총 출산 횟수","IVF 출산 횟수","DI 출산 횟수",
                      #  "난자 출처","정자 출처","난자 기증자 나이","정자 기증자 나이",
                      #  "동결 배아 사용 여부","신선 배아 사용 여부","기증 배아 사용 여부","대리모 여부"]
categorical_columns =  ["시술 시기 코드","시술 당시 나이", "시술 유형", "특정 시술 유형","배란 자극 여부",
                       "단일 배아 이식 여부","착상 전 유전 진단 사용 여부",
                       "남성 주 불임 원인","남성 부 불임 원인","여성 주 불임 원인","여성 부 불임 원인",
                       "부부 주 불임 원인","부부 부 불임 원인","불명확 불임 원인","불임 원인 - 난관 질환",
                       "불임 원인 - 남성 요인","불임 원인 - 배란 장애","불임 원인 - 자궁내막증","불임 원인 - 정자 농도",
                       "배아 생성 주요 이유","난자 출처","정자 출처","난자 기증자 나이","정자 기증자 나이",
                       "동결 배아 사용 여부","신선 배아 사용 여부","기증 배아 사용 여부","대리모 여부",
                        '배란 유도 유형',  '불임 원인 - 자궁경부 문제', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태']


for col in categorical_columns:
    # mode_value = X[col].mode()[0]
    # X[col].fillna(mode_value, inplace=True)  # 결측치 대체
    # test[col].fillna(mode_value, inplace = True)

    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

numeric_columns = ["총 시술 횟수","클리닉 내 총 시술 횟수","IVF 시술 횟수","DI 시술 횟수",
                       "총 임신 횟수","IVF 임신 횟수","DI 임신 횟수","총 출산 횟수","IVF 출산 횟수","DI 출산 횟수",
                   '임신 시도 또는 마지막 임신 경과 연수', "총 생성 배아 수","미세주입된 난자 수","미세주입에서 생성된 배아 수",
                   "이식된 배아 수","미세주입 배아 이식 수","저장된 배아 수","미세주입 후 저장된 배아 수",
                   "해동된 배아 수","해동 난자 수","수집된 신선 난자 수","저장된 신선 난자 수","혼합된 난자 수",
                   "파트너 정자와 혼합된 난자 수","기증자 정자와 혼합된 난자 수",
                   '난자 해동 경과일', "난자 혼합 경과일","배아 이식 경과일", '배아 해동 경과일']

# numeric_columns = ["총 생성 배아 수","미세주입된 난자 수","미세주입에서 생성된 배아 수",
#                    "이식된 배아 수","미세주입 배아 이식 수","저장된 배아 수","미세주입 후 저장된 배아 수",
#                    "수집된 신선 난자 수","혼합된 난자 수","파트너 정자와 혼합된 난자 수",
#                    "기증자 정자와 혼합된 난자 수","배아 이식 경과일"]

for col in numeric_columns:
    mode_value = X[col].mode()[0]  # 최빈값 구하기
    X[col].fillna(mode_value, inplace=True)  # 결측치 대체
    test[col].fillna(mode_value, inplace = True)

/var/folders/_2/7d52k4j5451f0ytscvrz28k40000gn/T/ipykernel_1352/502885209.py:47: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(mode_value, inplace=True)  # 결측치 대체
/var/folders/_2/7d52k4j5451f0ytscvrz28k40000gn/T/ipykernel_1352/502885209.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [76]:
# 분포 정규화 (왜도가 큰 변수에 대해 로그 변환)
skew_threshold = 1  # 왜도 임계값
for col in numeric_columns:
    skew_val = X[col].skew()
    if skew_val > skew_threshold:
        X[col] = np.log1p(X[col])
        test[col] = np.log1p(test[col])
        print(f"{col}: skewness {skew_val:.2f} -> 로그 변환 적용")

총 시술 횟수: skewness 1.10 -> 로그 변환 적용
클리닉 내 총 시술 횟수: skewness 1.51 -> 로그 변환 적용
IVF 시술 횟수: skewness 1.21 -> 로그 변환 적용
DI 시술 횟수: skewness 5.43 -> 로그 변환 적용
총 임신 횟수: skewness 2.34 -> 로그 변환 적용
IVF 임신 횟수: skewness 2.42 -> 로그 변환 적용
DI 임신 횟수: skewness 11.82 -> 로그 변환 적용
총 출산 횟수: skewness 2.52 -> 로그 변환 적용
IVF 출산 횟수: skewness 2.59 -> 로그 변환 적용
DI 출산 횟수: skewness 12.75 -> 로그 변환 적용
임신 시도 또는 마지막 임신 경과 연수: skewness 8.95 -> 로그 변환 적용
총 생성 배아 수: skewness 1.20 -> 로그 변환 적용
미세주입된 난자 수: skewness 1.52 -> 로그 변환 적용
미세주입에서 생성된 배아 수: skewness 1.77 -> 로그 변환 적용
저장된 배아 수: skewness 3.83 -> 로그 변환 적용
미세주입 후 저장된 배아 수: skewness 5.40 -> 로그 변환 적용
해동된 배아 수: skewness 5.19 -> 로그 변환 적용
해동 난자 수: skewness 18.56 -> 로그 변환 적용
저장된 신선 난자 수: skewness 15.87 -> 로그 변환 적용
혼합된 난자 수: skewness 1.00 -> 로그 변환 적용
파트너 정자와 혼합된 난자 수: skewness 1.04 -> 로그 변환 적용
기증자 정자와 혼합된 난자 수: skewness 6.11 -> 로그 변환 적용
난자 해동 경과일: skewness 358.01 -> 로그 변환 적용
난자 혼합 경과일: skewness 34.01 -> 로그 변환 적용
배아 해동 경과일: skewness 26.43 -> 로그 변환 적용


In [77]:
def make_feature(df):
  #df['난자 활용률'] = df['미세주입된 난자 수'] / (df['수집된 신선 난자 수'] + df['해동 난자 수'] + 1)
  df['미세주입 배아 생성률'] = df['미세주입에서 생성된 배아 수'] / (df['미세주입된 난자 수'] + 1)
  df['미세주입 배아 이식률'] = df['미세주입 배아 이식 수'] / (df['미세주입에서 생성된 배아 수'] + 1)
  df['배아 이식률'] = df['이식된 배아 수'] / (df['총 생성 배아 수'] + 1)
  df['저장 배아 비율'] = df['저장된 배아 수'] / (df['총 생성 배아 수'] + 1)
  df['해동 배아 생존율'] = df['해동된 배아 수'] / (df['저장된 배아 수'] + 1)
  df['파트너 정자 혼합 비율'] = df['파트너 정자와 혼합된 난자 수'] / (df['혼합된 난자 수'] + 1)
  #df['미세주입 배아 이식 비율'] = df['미세주입 배아 이식 수'] / (df['이식된 배아 수'] + 1)

  df['임신 성공률'] = df['총 임신 횟수'] / (df['총 시술 횟수'] + 1)
  df['IVF 시술 성공률'] = df['IVF 임신 횟수'] / (df['IVF 시술 횟수'] + 1)
  df['DI 시술 성공률'] = df['DI 임신 횟수'] / (df['DI 시술 횟수'] + 1)
  # df['출산율'] = df['총 출산 횟수'] / (df['총 임신 횟수'] + 1)
  # df['IVF 출산율'] = df['IVF 출산 횟수'] / (df['IVF 임신 횟수'] + 1)
  # df['DI 출산율'] = df['DI 출산 횟수'] / (df['DI 임신 횟수'] + 1)
  # df['시술 기준 출산 성공률'] = df['총 출산 횟수'] / (df['총 시술 횟수'] + 1)

  return df

In [78]:
X = make_feature(X)
test = make_feature(test)

X.shape, test.shape

((256351, 71), (90067, 71))

In [79]:
test_pred_total = []
scores = []
cv = StratifiedKFold(n_splits = 10, random_state = 42, shuffle=True)

for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    print('='*50)
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostClassifier(auto_class_weights='Balanced', eval_metric='AUC', random_state=42)
    model.fit(
        X_train,
        y_train,
        cat_features=categorical_columns,
        early_stopping_rounds= 300,
        eval_set=[(X_val,y_val)],
        verbose = 100
        )

    scores.append(model.get_best_score()["validation"]["AUC"])

    test_pred = model.predict_proba(test)[:, 1]
    test_pred_total.append(test_pred)


Learning rate set to 0.121504
0:	test: 0.7191414	best: 0.7191414 (0)	total: 76.3ms	remaining: 1m 16s
100:	test: 0.7424009	best: 0.7424262 (99)	total: 7.32s	remaining: 1m 5s
200:	test: 0.7427889	best: 0.7430931 (155)	total: 15.4s	remaining: 1m 1s
300:	test: 0.7426165	best: 0.7430931 (155)	total: 24s	remaining: 55.7s
400:	test: 0.7423939	best: 0.7430931 (155)	total: 31.8s	remaining: 47.6s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.7430930921
bestIteration = 155

Shrink model to first 156 iterations.
Learning rate set to 0.121504
0:	test: 0.7117283	best: 0.7117283 (0)	total: 69.4ms	remaining: 1m 9s
100:	test: 0.7318983	best: 0.7319154 (97)	total: 7.45s	remaining: 1m 6s
200:	test: 0.7328416	best: 0.7328537 (199)	total: 14.9s	remaining: 59.3s
300:	test: 0.7322176	best: 0.7328537 (199)	total: 22.5s	remaining: 52.2s
400:	test: 0.7321369	best: 0.7328537 (199)	total: 29.9s	remaining: 44.7s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.7328537137


In [80]:
np.mean(scores) 

0.7400397883458353

In [81]:
test_pred_total = np.mean(test_pred_total, axis=0)

In [82]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = test_pred_total

In [84]:
sample_submission.to_csv('./sunghyun_catboost+log.csv', index=False)